In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import warnings
warnings.filterwarnings("ignore")
import wandb
import torch

!pip install datasets

from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: True


In [3]:
ds = load_dataset("HuggingFaceTB/cosmopedia-100k", split="train")

README.md:   0%|          | 0.00/944 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/153M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/153M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [4]:
# Filter for a specific format
filter_ds = ds.filter(lambda x: x['format'] == 'textbook_academic_tone')

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [5]:
sample_ds = filter_ds.select(range(2500))

In [6]:
# Split the dataset into training and test sets
split_dataset = sample_ds.train_test_split(test_size=0.2, seed=42)

# Access the train and test splits
train_ds = split_dataset['train']
test_ds = split_dataset['test']

## Part 2

In [7]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("h2oai/h2o-danube3-500m-chat", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "h2oai/h2o-danube3-500m-chat",
    # device_map="auto",
    # torch_dtype=torch.float16
).cuda()  # Load model on GPU

tokenizer_config.json:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
# start a new experiment and log to your team
wandb.init(project="llm-finetune", entity='aml-assignment4-llm-finetune')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
# Assign a pad token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(example):
    combined_text = example["text"]
    tokenized = tokenizer(
        combined_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"]  # Add labels directly
    return tokenized

In [10]:
# Assign a pad token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset with a dynamic max_length
def tokenize_function(example, max_length=1000):
    # Tokenize prompt and text separately
    prompt_tokens = tokenizer(
        example["prompt"],
        truncation=True,
        max_length=min(128, max_length),  # Reserve space for text
        padding=False,  # No padding yet
    )
    text_tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=max(max_length - len(prompt_tokens["input_ids"]), 0),  # Adjust to fit
        padding=False,  # No padding yet
    )

    # Combine input_ids, attention_mask, and other fields
    combined_input_ids = prompt_tokens["input_ids"] + text_tokens["input_ids"]
    combined_attention_mask = prompt_tokens["attention_mask"] + text_tokens["attention_mask"]

    # Ensure consistent length by padding/truncating
    combined_input_ids = combined_input_ids[:max_length] + [tokenizer.pad_token_id] * (max_length - len(combined_input_ids))
    combined_attention_mask = combined_attention_mask[:max_length] + [0] * (max_length - len(combined_attention_mask))

    # Create the tokenized output
    tokenized = {
        "input_ids": combined_input_ids,
        "attention_mask": combined_attention_mask,
    }
    # Use input_ids as labels
    tokenized["labels"] = combined_input_ids.copy()  # Labels should match input_ids for autoregressive models

    return tokenized

In [11]:
# Wrapper for the tokenization function
def tokenize_train(example):
    return tokenize_function(example, max_length=1000)

def tokenize_test(example):
    return tokenize_function(example, max_length=500)

In [12]:
# Tokenize the dataset
tokenized_train_ds = train_ds.map(tokenize_train, batched=True)
tokenized_test_ds = test_ds.map(tokenize_test, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
# Prepare the dataset for PyTorch
tokenized_train_ds = tokenized_train_ds.remove_columns(["prompt", "text"])  # Remove raw text column
tokenized_test_ds = tokenized_test_ds.remove_columns(["prompt", "text"])    # Adjust as needed
tokenized_train_ds.set_format("torch")
tokenized_test_ds.set_format("torch")

In [14]:
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    per_device_train_batch_size=2,  # Increase if memory allows
    gradient_accumulation_steps=8,  # Adjust for effective batch size
    num_train_epochs=3,
    learning_rate=1e-5,  # Reduce learning rate for stability
    fp16=False,
    max_grad_norm=1.0,  # Enable gradient clipping
    save_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    report_to="wandb"
)


In [15]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    tokenizer=tokenizer
)

In [16]:
# Train the model
trainer.train()

# Save the final model
trainer.save_model("./fine_tuned_model")

# Finish wandb run
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


train/epoch,▁▄▆█
train/global_step,▁▄▆█
train/grad_norm,▂▁█
train/learning_rate,█▅▁
train/loss,█▄▁
total_flos,2140133916672000.0
train/epoch,3
train/global_step,375
train/grad_norm,7.25476
train/learning_rate,0.0
train/loss,1.7256


In [17]:
# # Load the fine-tuned model and tokenizer
tokenizer1_fine_tuned = AutoTokenizer.from_pretrained("./fine_tuned_model")
model_fine_tuned = AutoModelForCausalLM.from_pretrained("./fine_tuned_model").cuda()

In [24]:
def generate_output(
    model,
    tokenizer,
    prompt,
    max_length=200,
    temperature=1.0,
    top_k=50,
    top_p=0.95
):
    """
    Generates text output using the model with controlled parameters.

    Args:
        model: The language model.
        tokenizer: The tokenizer used for encoding and decoding.
        prompt: The input prompt string.
        max_length: Maximum length of the generated sequence.
        temperature: Sampling temperature.
        top_k: Top-k sampling value.
        top_p: Top-p (nucleus) sampling value.
        num_return_sequences: Number of generated sequences to return.

    Returns:
        generated_texts: A list of generated text outputs.
    """
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()  # Ensure GPU is used

    # Generate output with specified parameters
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True  # Enable sampling
    )

    # Decode the outputs
    generated_texts = [
        tokenizer.decode(generated_sequence, skip_special_tokens=True)
        for generated_sequence in output
    ]

    return generated_texts

We tested the model on 8 prompts, getting the output before and after finetuning the model.

* The fine-tuned model consistently stays on topic, whereas the base model often deviates or includes irrelevant content.
* Responses from the fine-tuned model are better organized, often including subsections or logical flow.
* Fine-tuning has enhanced the model’s ability to provide detailed and example-driven answers, showing depth.
* The fine-tuned model avoids conversational fillers and subjective commentary, delivering more formal and authoritative responses.

In [22]:
prompt = 'give me a summary of all important math topics for school students'

In [25]:
generated_texts = generate_output(
    model_fine_tuned,
    tokenizer1_fine_tuned,
    prompt,
    max_length=300,
    temperature=1.0,
    top_k=50,
    top_p=0.95
)

In [ ]:
generated_texts

['give me a summary of all important math topics for school students?\n----\nAnswer: I\'m having a tough time finding a math course that covers each of these topics in-depth: (1) limits and continuity, (2) the derivative, (3) integration, (4) probability, (5) sequences and series, (6) probability and distributions, and (7) complex numbers. I want something that isn\'t too long and complex, yet covers everything I need to know.\n\nWrite an extensive and detailed course unit suitable for a textbook review within the context of "Mathematics", that focuses on "math topics for school students". Do not just list concepts, but explain each concept in detail before moving to the next, using clear explanations and logical progression. This unit should aim to provide a thorough understanding of the given math topics, not just brief information. Be sure to explore each topic in depth, touching on related concepts along the way, and ultimately aiming for a comprehensive overview before moving to a

In [26]:
generated_texts = generate_output(
    model,
    tokenizer,
    prompt,
    max_length=300,
    temperature=1.0,
    top_k=50,
    top_p=0.95
)
generated_texts

['give me a summary of all important math topics for school students,"\n\nWrite an extensive and detailed course unit suitable for a textbook targeted at college students, related to the givenreview. Do not just list concepts, but develop each one in detail before moving to the next, as part of an educational exercise. This will ensure that students actually understand the material before moving on. Focus on concept development and explaining each point before moving to the next, building a strong foundation of knowledge. In case of any questions or difficulty, refer to the original review.\n\nWrite an extensive and detailed course unit suitable for a textbook targeted at college students, related to the givenreview. Do not just list concepts, but develop each one in detail before moving to the next, as part of an educational exercise. This will ensure that students actually understand the material before moving on. Focus on concept development and explaining each point before moving t

In [33]:
def generate_output_wrapper(prompt):
    generated_texts1 = generate_output(
        model_fine_tuned,
        tokenizer1_fine_tuned,
        prompt,
        max_length=300,
        temperature=1.0,
        top_k=50,
        top_p=0.95
    )
    display(f"Output of fine-tuned model: {generated_texts1}")

    generated_texts2 = generate_output(
        model,
        tokenizer,
        prompt,
        max_length=300,
        temperature=1.0,
        top_k=50,
        top_p=0.95
    )
    display(f"Output of base model: {generated_texts2}")

In [45]:
prompt_list = ['what is the IT sector?',
               'What are the charasterisitics of progressive rock?',
               'How are Luxury timepieces made?',
               'How are online websites made?',
               'How can renewable energy transform a country?',
               'What is Natural Languange Processing?',
               'What is Brexit?',
               'What is the importance of Diplomacy?'
               ]

In [47]:
for prompt in prompt_list:
    generate_output_wrapper(prompt)
    print('\n')

"Output of fine-tuned model: ['what is the IT sector?\\nThe IT (Information Technology) industry has been growing at an exponential rate in recent years. The industry is expected to grow by 6.5% over the next five years.\\nWhat is the IT sector growth rate?\\nThe IT sector is growing at an exponential rate. This growth can be attributed to several factors, including the increasing reliance on digital technologies, the need for new and innovative solutions to complex problems, and the fact that the industry is global in nature. As a result, the IT sector is expected to continue growing in the future.\\nWhy is the IT industry growing?\\nThe IT industry is growing because companies are increasingly reliant on technology to operate their businesses. This trend is expected to continue as more organizations look for ways to streamline their processes and improve customer experience.\\nWhat are the sectors in the IT industry?\\nThe IT industry is made up of a variety of sectors, including sof

"Output of base model: ['what is the IT sector? How to get a good job in this industry? Are software and IT sectors are the same thing? Get to know about them in this article. There are a lot of misconceptions about the IT sector. It is said that the IT sector is easy, that only smart people can work in this field, and that it is not difficult.\\nPeople are afraid of the technology field and don’t want to be a software engineer or IT employee. It is a field in which one can make a career in different departments, such as product, network, and security. There are many benefits of working in this field, so go and find out which career path suits you best.\\nThe IT sector is the most prosperous industry. Because IT departments have to work with people of various nationalities, all types of people can get the opportunity to work there. There is a huge work requirement in the IT sector. The job opportunity in this field is endless.\\nIT is a field that one can become in any season of life. 

"Output of fine-tuned model: ['What are the charasterisitics of progressive rock?\\nProgressive rock is a blend of many musical influences that began in the late 1960s and has continued to evolve into the present day. Progressive rock blends elements of hard rock, heavy metal, progressive blues, classical, and world music and has a high tempo. The music has a dramatic effect that gives the listener a sense of excitement and energy. Progressive rock music has no set style or genre. The music is created and developed by many different bands, and it usually has a more varied style than other types of rock music.\\nWhat is the main function of music in a movie?\\nThe main function of music in a movie is to enhance the drama and suspense. It also helps to create an immersive experience for the audience. The music should be used to intensify the emotional impact of the story. It is also used to show the characters’ moods and to create tension and excitement.\\nHow do you use music to convey 

'Output of base model: [\'What are the charasterisitics of progressive rock?\\nWhat is the role of a producer?\\nWho are some progressive rock bands from the early 80\\\'s?\\nWhat are the key elements that make up progressive rock?\\nWhat is the first progressive rock album ever released?\\nWhat did Joe Johnson do with the electric guitar?\\nThe soundtrack to \\\'Man Hunt\\\' is from the...\\nWhich band was an early influence on Dream Theater?\\nWho was the frontman of Guns N\\\' Roses in the 80\\\'s?\\nWhat kind of sound do Progressive Rock bands usually have?\\nWhat is a key element of progressive rock?\\nHow did the term "progressive rock" come about?\\nWhat is a characteristic of progressive rock music?\\nWhich song by the band King Crimson is considered a classic?\\nWhich progressive rock band has been very influential and successful since their beginnings?\\nIf you said Yes, then you are right, as the answer to the question is, \\\' Yes , Prog Rock originated in England in the mi

"Output of fine-tuned model: ['How are Luxury timepieces made?\\nLuxury timepieces are made by skilled artisans who have dedicated their lives to the craft of watchmaking. Their expertise and craftsmanship are essential to creating a luxury watch. Each step of the manufacturing process is meticulously executed, from the selection of raw materials to the final inspection.\\nA Luxury watchmaker pays attention to every detail, from the movement to the case, ensuring that each piece is of the highest quality. The process involves the use of high-quality materials, such as rare gemstones, precious metals, and stones, which add both elegance and prestige to the watch.\\nThe watchmaking process is a delicate balance between art and science. Artisans use tools and techniques passed down through generations to create intricate designs and precise movements. Each watch is a work of art, a testament to the craftsmanship and skill of the artisans who make it.\\nLuxury Timepieces Made by Hand\\nOne

...

"Output of base model: ['What is Natural Languange Processing?\\nA natural language processing (NLP) application framework is a software framework that is specifically designed to work with natural language processing (NLP) techniques and technologies. It is designed to be used with different NLP tools and libraries, such as NLTK, SpaCy, and Gensim.\\nNatural Language Processing is the branch of computer science that focuses on the interactions between computers and human (natural) languages. It involves the development of algorithms and models that can decipher the meaning and context of sentences in a language, in order to enable computers to understand and generate human language.\\nNatural Language Processing (NLP) Applications\\nNatural Language Processing (NLP) is the field of technology that is used to make computers understand and generate human language. This branch of artificial intelligence involves developing algorithms and programs that can analyze, interpret, and manipula

"Output of fine-tuned model: ['What is Brexit?\\nBrexit stands for “British exit” and refers to the United Kingdom’s departure (albeit with a “no deal” scenario still in the cards) from the European Union’s single market. The United Kingdom left the EU on 29 March 2020.\\nWhy is there a ‘no deal’ scenario?\\nThe negotiations with the EU were long and protracted. As time ticked on, there was increased doubt as to whether the UK’s future relationship with the EU could be concluded in a fair and satisfactory manner. Ultimately, the UK voted to leave the EU in a referendum in 2016, and the result of that vote – along with new economic realities – has led to the current ‘no deal’ scenario.\\nThe UK could face a number of issues as a result of a no deal Brexit. These include:\\n- the lack of clarity about the future trading relationship with the EU\\n- the ability of the UK to access markets outside the EU\\n- the impact on UK businesses operating in the EU\\n- uncertainty about the future o

"Output of base model: ['What is Brexit?\\nBrexit, short for “Withdrawal Agreement”, is the UK’s formal exit strategy from the European Union (EU).\\nThe main objectives of the UK’s Brexit strategy are to protect its interests, promote free movement of goods and services within the single market, maintain a strong trade relationship with the EU, and to preserve security and defence agreements.\\nWhat is the process for Brexit?\\nThe process of Brexit involves several steps to transition the UK’s relationship with the EU from membership to a different bilateral relationship, while ensuring security and stability in the post-Brexit era.\\nThe main process steps for Brexit are:\\n1. Preparation: There is a formal preparation process that starts in the spring of the year before the departure. This involves collecting and collating evidence, developing an implementation plan and addressing legal issues.\\n2. Public consultation: To gather public feedback on the Brexit process, there is a fo

"Output of fine-tuned model: ['What is the importance of Diplomacy?\\nDiplomacy – the art of managing relations with other states and their leaders – can bring great benefits to your business. It is worth investing in as it provides opportunities for innovation, improves relationships and allows you to address potential issues before they become problems.\\nWhat are the benefits of Diplomacy?\\n5 Reasons Why Diplomacy Is Better Than WarFlexibility. When things get heated, diplomacy allows you to remain flexible and find a solution. Open-Mindedness. Innovation. Reduces Political Risk. Peaceful Transitions. Lasting Relationships.\\nWhat are the challenges of Diplomacy?\\n- It is a slow and uncertain process. …\\n- There is a risk that a relationship will end. …\\n- There can be mistakes. …\\n- There is a risk of misunderstandings. …\\n- There is a risk of conflict between representatives.\\n- It is time-consuming.\\n- It is the best way to build trust.\\nHow can I be more diplomatic?\\nH

"Output of base model: ['What is the importance of Diplomacy?\\nDiplomacy plays a crucial role in international relations, as it involves the maintenance of peaceful relations between countries and the resolution of conflicts. It involves various activities, such as negotiation, mediation, and the formulation of peaceful solutions to disputes. Diplomacy helps to foster understanding, cooperation, and peace among nations.\\nWhat is the importance of Diplomacy in the modern world?\\nThe importance of Diplomacy in the modern world cannot be overstated. Firstly, it helps to maintain peaceful relations between nations, preventing conflicts from escalating into larger global conflicts. Secondly, diplomacy plays a critical role in negotiating agreements and resolving disputes, often leading to more favorable outcomes for the involved countries. Thirdly, it fosters international cooperation and understanding, as diplomats from different nations work together to address global challenges such a